# ETL Vnstock data

In [1]:
#Load library
from stock_data import extract_stock_data, create_period
from vnstock3 import Vnstock
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import schedule
import time as tm
import pyodbc as odbc
from sqlalchemy import create_engine

# Extract data from vnstock

In [20]:
company = ['acb','bid','ctg','eib','hdb','lpb','mbb','msb','ocb','stb','tcb','tpb','vcb','vib','vpb']
def extract_data(company):
    financial_statement, historical_price = extract_stock_data(company)
    financial_statement = financial_statement.loc[:, ~financial_statement.columns.duplicated(keep='first')]
    financial_statement['Date'] = financial_statement.apply(lambda row: create_period(row['yearReport'], row['lengthReport']), axis=1)
    financial_statement['Graham value'] = (22.5*financial_statement['EPS (VND)'] * financial_statement['BVPS (VND)'])**(1/2)
    financial_statement = financial_statement.drop(columns=['Net other income/expenses'])
    financial_statement.to_csv(r'C:\Users\LENOVO\Downloads\financial_statment.csv')
    historical_price.to_csv(r'C:\Users\LENOVO\Downloads\history_price.csv')
    return financial_statement, historical_price

In [24]:
# Variables to store the results
financial_statement = None
historical_price = None

# Function to call `extract_data` and assign results
def scheduled_extract_data():
    global financial_statement, historical_price
    financial_statement, historical_price = extract_data(company)
    print("Data extraction complete!")

In [21]:
financial_statement, historical_price = extract_data(company)

2024-09-11 22:55:13,631 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:18,448 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:22,159 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:25,942 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:29,996 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:44,741 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:48,441 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 22:55:52,160 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yế

In [18]:
financial_statement.head()

,ticker,yearReport,lengthReport,Net Inventories,Other current assets,Investment in properties,Other non-current assets,Other Reserves,Budget sources and other funds,MINORITY INTERESTS,...,Days Inventory Outstanding,Days Payable Outstanding,Cash Cycle,Inventory Turnover,(ST+LT borrowings)/Equity,Debt/Equity,Fixed Asset-To-Equity,Owners' Equity/Charter Capital,Date,Graham value
0,ACB,2024,2,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,9.290830,0.062380,1.674466,20240630,19414.372725
1,ACB,2024,1,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,8.724440,0.062450,1.674423,20240331,18149.567315
2,ACB,2023,4,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,9.130151,0.067120,1.588569,20231231,20590.715852
3,ACB,2023,3,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,8.704559,0.066554,1.496091,20230930,20060.296430
4,ACB,2023,2,0.0,0.0,177005000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,9.035334,0.068278,1.407477,20230630,19038.848373


In [7]:
historical_price.head()

,time,open,high,low,close,volume,Ma CK
0,2013-01-02 07:00:00,2.73,2.91,2.73,2.90,2578600,ACB
1,2013-01-03 07:00:00,2.93,2.98,2.80,2.85,1967200,ACB
2,2013-01-04 07:00:00,2.83,2.88,2.80,2.88,1170100,ACB
3,2013-01-07 07:00:00,2.85,2.93,2.85,2.88,1355700,ACB
4,2013-01-08 07:00:00,2.86,2.93,2.83,2.90,2180000,ACB


# Import data into Azure SQL database

In [10]:
# Connection string
server = 'sqldbstock.database.windows.net'
database = 'VNstock'
username = 'QuangThien'
password = '0832212568Tu'

In [28]:
def import_data(server, database, username, password, fs, hc):
    connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    # Create the SQLAlchemy engine
    engine = create_engine(connection_string)
    #Import three dataframe into three tables
    # Create and insert data into 'financial_statement' table
    financial_statement.to_sql('financial_statement', con=engine, if_exists='replace', index=False)
    
    # Create and insert data into 'history_price' table
    historical_price.to_sql('history_price', con=engine, if_exists='replace', index=False)
    
    print("DataFrames have been successfully imported into the SQL Server database.")

In [ ]:
schedule.every().day.at('08:00').do(scheduled_extract_data)
schedule.every().day.at('08:00').do(import_data, server = server, database = database, username = username, password = password, fs = financial_statement, hc = historical_price)

while True:
    schedule.run_pending()
    tm.sleep(1)

2024-09-11 23:22:32,202 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:35,524 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:38,836 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:42,354 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:45,622 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:48,986 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:52,299 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:22:56,280 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yế

Data extraction complete!
DataFrames have been successfully imported into the SQL Server database.


2024-09-11 23:24:34,124 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:24:39,122 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:24:45,368 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:24:50,561 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:24:56,201 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:25:01,293 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:25:06,674 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:25:11,938 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yế

Data extraction complete!
DataFrames have been successfully imported into the SQL Server database.


2024-09-11 23:26:56,662 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:01,359 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:06,834 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:11,894 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:15,506 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:20,628 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:25,144 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:27:29,360 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yế

Data extraction complete!
DataFrames have been successfully imported into the SQL Server database.


2024-09-11 23:29:09,274 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:13,938 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:17,268 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:20,768 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:24,695 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:28,403 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:31,971 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-09-11 23:29:35,285 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yế